<a href="https://colab.research.google.com/github/rkp74/Myindiaa/blob/main/ContentGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Template Based Generation

## OpenAI API

In [1]:
!pip install langchain-community --quiet
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [3]:
import getpass

# Prompt the user to input their API key
api_key = getpass.getpass('Please enter your OpenAI API key: ')

Please enter your OpenAI API key: ··········


In [4]:
os.environ['OPENAI_API_KEY'] = api_key

In [5]:
# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["attributes"],
    template="Generate a product description based on the following attributes:\n{attributes}"
)

In [6]:
# Define a function to generate product descriptions
def generate_product_description(attributes, model_name="gpt-3.5-turbo"):
    """
    Generate a product description using OpenAI's GPT model based on provided product attributes.

    :param attributes: Dictionary containing product attributes
    :param model_name: Name of the OpenAI model to use
    :return: Generated product description as a string
    """
    # Convert attributes dictionary to a formatted string
    attributes_str = "\n".join(f"{key}: {value}" for key, value in attributes.items())

    # Initialize the OpenAI model for chat
    llm = ChatOpenAI(model=model_name)

    # Create a RunnableSequence
    sequence = RunnableSequence(prompt_template | llm)

    # Generate the product description
    description = sequence.invoke({"attributes": attributes_str})

    return description

In [7]:
# Example product attributes
product_attributes = {
    "Product Name": "Eco-Friendly Water Bottle",
    "Material": "Stainless Steel",
    "Capacity": "750ml",
    "Color": "Ocean Blue",
    "Features": "BPA-free, Leak-proof, Insulated",
    "Target Audience": "Outdoor Enthusiasts, Eco-conscious Consumers",
    "Brand": "GreenLife"
}

In [8]:
# Generate the product description using a specific model
description = generate_product_description(product_attributes, model_name="gpt-3.5-turbo")
print("Generated Product Description (gpt-3.5-turbo):\n")
print(description)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Generated Product Description (gpt-3.5-turbo):

content='Introducing the GreenLife Eco-Friendly Water Bottle, the perfect companion for outdoor enthusiasts and eco-conscious consumers. Made from high-quality stainless steel in a stunning ocean blue color, this 750ml bottle is both stylish and sustainable.\n\nStay hydrated on-the-go with confidence, as our water bottle is BPA-free and leak-proof, ensuring a mess-free experience every time. The insulated design keeps your drinks at the perfect temperature for hours, making it ideal for hiking, camping, or simply enjoying a day outdoors.\n\nChoose the GreenLife Eco-Friendly Water Bottle and make a positive impact on the environment while staying hydrated in style.' response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 71, 'total_tokens': 196}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-47d14711-8133-465d-9207-3d53fd921ced-0'


## HuggingFace Transformers

In [11]:
pip install -U langchain-huggingface transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.3 MB/s eta 0:00:00


In [18]:
import os
import torch
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnableSequence
from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint
from transformers import pipeline

In [13]:
import getpass

# Prompt the user to input their API key
hf_api_key = getpass.getpass('Please enter your Hugging Face API key: ')

Please enter your Hugging Face API key: ··········


In [14]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_api_key


In [20]:
# Define a function to generate product descriptions using Hugging Face Transformers
def generate_product_description_hf(attributes, model_name="EleutherAI/gpt-neo-2.7B"):
    """
    Generate a product description using Hugging Face Transformers based on provided product attributes.

    :param attributes: Dictionary containing product attributes
    :param model_name: Hugging Face model to use
    :return: Generated product description as a string
    """
    # Convert attributes dictionary to a formatted string
    attributes_str = "\n".join(f"{key}: {value}" for key, value in attributes.items())

    # Create the prompt
    prompt = f"Generate a detailed and engaging product description for the following product based on its attributes:\n\n{attributes_str}\n\nProduct Description:"

    # Load the Hugging Face model and tokenizer
    generator = pipeline('text-generation', model=model_name, device=0 if torch.cuda.is_available() else -1)

    # Generate the product description
    response = generator(prompt, max_length=300, num_return_sequences=1, do_sample=True, truncation=True)

    # Extract the generated text
    description = response[0]['generated_text']

    # Remove the prompt from the generated text
    description = description.replace(prompt, "").strip()

    return description

In [21]:
# Example product attributes
product_attributes = {
    "Product Name": "Eco-Friendly Water Bottle",
    "Material": "Stainless Steel",
    "Capacity": "750ml",
    "Color": "Ocean Blue",
    "Features": "BPA-free, Leak-proof, Insulated",
    "Target Audience": "Outdoor Enthusiasts, Eco-conscious Consumers",
    "Brand": "GreenLife"
}

In [22]:
# Generate the product description using Hugging Face Transformers
description_hf = generate_product_description_hf(product_attributes, model_name="EleutherAI/gpt-neo-2.7B")
print("Generated Product Description:\n", description_hf)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Product Description:
 A water bottle is designed to carry as much liquid as possible by its height, so that the user can drink or take a drink with a smooth taste and feel, regardless of whether the user is running, cycling, hiking, mountain climbing or the like. The size and shape of a water bottle can be different, and the material of the bottle can be stainless steel, aluminum, plastic, glass, etc. A water bottle is widely used in sports and outdoor, and many water bottles are water bottles for sports. According to the size and shape of the bottle, the product can be divided into a large water bottle for large use and a water bottle for small use. At the same time, the function of water bottles is to carry out drink, so the type of water bottles for use varies from small water bottles for drinking to large water bottles for taking a drink. By being divided into two types, the product has good value added functions of being light, water-tight, long-lasting and low-maintenan

# Rule Based Generation

In [24]:
import random

In [25]:
# Rule-based content generation
def generate_content_rule_based(attributes):
    product_name = attributes.get("Product Name", "")
    material = attributes.get("Material", "")
    capacity = attributes.get("Capacity", "")
    color = attributes.get("Color", "")
    features = attributes.get("Features", "")
    audience = attributes.get("Target Audience", "")
    brand = attributes.get("Brand", "")

    template = f"Introducing the {brand} {product_name}, a {capacity} {material} bottle in {color}. Perfect for {audience}. Features include {features}."

    return template

In [26]:
# Example product attributes
product_attributes = {
    "Product Name": "Eco-Friendly Water Bottle",
    "Material": "Stainless Steel",
    "Capacity": "750ml",
    "Color": "Ocean Blue",
    "Features": "BPA-free, Leak-proof, Insulated",
    "Target Audience": "Outdoor Enthusiasts, Eco-conscious Consumers",
    "Brand": "GreenLife"
}

In [27]:
rule_based_content = generate_content_rule_based(product_attributes)

In [28]:
print("Rule-Based Content Generation:")
print(rule_based_content)

Rule-Based Content Generation:
Introducing the GreenLife Eco-Friendly Water Bottle, a 750ml Stainless Steel bottle in Ocean Blue. Perfect for Outdoor Enthusiasts, Eco-conscious Consumers. Features include BPA-free, Leak-proof, Insulated.


# Neural Network Based (LSTM)

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [30]:
# Sample product attributes (replace with your actual data)
product_attributes = [
    {"color": "blue", "material": "stainless steel", "capacity": "750ml"},
    {"color": "green", "material": "plastic", "capacity": "500ml"},
    {"color": "red", "material": "glass", "capacity": "1 liter"}
]

In [31]:
# Function to tokenize attributes
def tokenize_attributes(attributes):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(attributes)
    total_words = len(tokenizer.word_index) + 1  # Add 1 for the padding token
    return tokenizer, total_words

In [32]:
# Function to generate sequences from attributes
def generate_sequences(tokenizer, attributes):
    sequences = tokenizer.texts_to_sequences([f"{key}: {value}" for attr in attributes for key, value in attr.items()])
    return sequences

In [33]:
# Function to pad input sequences
def pad_input_sequences(sequences):
    max_sequence_len = max([len(seq) for seq in sequences])
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')
    return padded_sequences, max_sequence_len

In [34]:
# Function to build LSTM model
def build_lstm_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))  # Embedding layer
    model.add(LSTM(150))  # LSTM layer with 150 units
    model.add(Dense(total_words, activation='softmax'))  # Dense output layer with softmax activation
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [35]:
# Function to train LSTM model
def train_lstm_model(model, xs, ys, epochs=100):
    model.fit(xs, ys, epochs=epochs, verbose=1)
    return model


In [36]:
# Function to generate text based on trained model
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    generated_text = seed_text
    for _ in range(next_words):
        token_list = seed_text.split()[1:]  # Remove "Attributes:" and take the rest as tokens
        token_list = tokenizer.texts_to_sequences([" ".join(token_list)])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Predict probabilities for each word
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Get the index of the word with the highest probability
        predicted_index = np.argmax(predicted_probs)

        # Convert index to word
        predicted_word = None
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                predicted_word = word
                break

        # Handle special case for non-integer tokens (product attribute values)
        if predicted_word is None:
            predicted_word = token_list[0][0]  # Use the first token as the predicted word

        generated_text += " " + str(predicted_word)

    # Convert the tokenized indices back to text
    final_text = "Attributes:"
    for token in generated_text.split()[1:]:
        try:
            if token.isdigit():
                word = tokenizer.index_word[int(token)]
            else:
                word = token
            final_text += f" {word}"
        except (KeyError, ValueError):
            final_text += f" {token}"

    return final_text



In [37]:
# Function to generate product descriptions
def generate_product_descriptions(attributes, tokenizer, model, max_sequence_len):
    sequences = generate_sequences(tokenizer, attributes)
    padded_sequences, _ = pad_input_sequences(sequences)

    descriptions = []
    for seq in padded_sequences:
        seed_text = "Attributes:"
        for token in seq:
            seed_text += f" {token}"
        generated_text = generate_text(seed_text, 20, model, max_sequence_len, tokenizer)
        descriptions.append(generated_text)

    return descriptions

In [38]:
# Main script
def main():
    # Tokenize attributes
    tokenizer, total_words = tokenize_attributes([f"{key}: {value}" for attr in product_attributes for key, value in attr.items()])

    # Generate sequences and pad input
    sequences = generate_sequences(tokenizer, product_attributes)
    padded_sequences, max_sequence_len = pad_input_sequences(sequences)

    # Create predictors and labels
    xs, labels = padded_sequences[:,:-1], padded_sequences[:,-1]
    ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

    # Build and train LSTM model
    lstm_model = build_lstm_model(total_words, max_sequence_len)
    lstm_model = train_lstm_model(lstm_model, xs, ys)

    # Generate product descriptions
    descriptions = generate_product_descriptions(product_attributes, tokenizer, lstm_model, max_sequence_len)

    # Print generated descriptions
    for i, desc in enumerate(descriptions):
        print(f"Generated Description {i+1}: {desc}")

if __name__ == "__main__":
    main()


Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 2.7088 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 2.6823 - accuracy: 0.2222
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 2.6471 - accuracy: 0.3333
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 2.5958 - accuracy: 0.3333
Epoch 5/100
1/1 [==============================] - 0s 15ms/step - loss: 2.5247 - accuracy: 0.3333
Epoch 6/100
1/1 [==============================] - 0s 20ms/step - loss: 2.4335 - accuracy: 0.3333
Epoch 7/100
1/1 [==============================] - 0s 15ms/step - loss: 2.3277 - accuracy: 0.3333
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 2.2205 - accuracy: 0.3333
Epoch 9/100
1/1 [==============================] - 0s 16ms/step - loss: 2.1257 - accuracy: 0.3333
Epoch 10/100
1/1 [==============================] - 0s 15ms/step - loss: 2.0428 - accuracy: 0.4444
Epoch 11/100
1/1 

# A/B Testing

In [23]:
# Hypothetical results
results = {
    "Description 1": {"CTR": 0.05, "Conversion Rate": 0.02, "Engagement": 50},
    "Description 2": {"CTR": 0.07, "Conversion Rate": 0.03, "Engagement": 70},
    "Description 3": {"CTR": 0.04, "Conversion Rate": 0.01, "Engagement": 40},
    "Description 4": {"CTR": 0.06, "Conversion Rate": 0.025, "Engagement": 60}
}

# Determine the best performing description
best_description = max(results, key=lambda k: (results[k]["CTR"], results[k]["Conversion Rate"], results[k]["Engagement"]))
print(f"The best performing description is: {best_description} with metrics: {results[best_description]}")


The best performing description is: Description 2 with metrics: {'CTR': 0.07, 'Conversion Rate': 0.03, 'Engagement': 70}
